In [1]:
import utils, random, copy, statistics
from ucimlrepo import fetch_ucirepo
import pandas as pd
import numpy as np
from config import config
from itertools import combinations, permutations, product
from pgmpy.models import BayesianNetwork
from pgmpy.estimators import BicScore
import networkx as nx
from castle.algorithms import PC
from scipy.spatial.distance import pdist, squareform
from python_tsp.exact import solve_tsp_dynamic_programming
from sentence_transformers import SentenceTransformer
from openai import OpenAI

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /Users/chunyuko/.cache/huggingface/token
Login successful


INFO:root:You can use `os.environ['CASTLE_BACKEND'] = backend` to set the backend(`pytorch` or `mindspore`).
INFO:root:You are using ``pytorch`` as the backend.


In [17]:
import pandas as pd
from castle.common import GraphDAG
from castle.metrics import MetricsDAG
from castle.datasets import DAG, IIDSimulation
from castle.algorithms import PC
from castle.common.priori_knowledge import PrioriKnowledge

In [31]:
method = 'linear'
sem_type = 'gauss'
n_nodes = 10
n_edges = 15
n = 2000

weighted_random_dag = DAG.erdos_renyi(n_nodes=n_nodes, n_edges=n_edges, weight_range=(0.5, 2.0), seed=1)
dataset = IIDSimulation(W=weighted_random_dag, n=n, method=method, sem_type=sem_type)
true_dag, X = dataset.B, dataset.X

INFO:root:Finished synthetic dataset


In [34]:
X = pd.DataFrame(X)

In [51]:
labels = X.columns.tolist()

In [62]:
[*product([0,1])]

[(0,), (1,)]

In [47]:
pc = PC(variant="original")
pc.learn(shuffled_x)
graph = pc.causal_matrix
network = nx.DiGraph(graph)

In [48]:
def get_undirected_edges(graph):
    undirected_edges = set()
    for edge in graph.edges():
        if graph.has_edge(*edge[::-1]):
            undirected_edges.add(frozenset(edge))
    return [(u, v) for u, v in undirected_edges]

undirected_edges = get_undirected_edges(network)
undirected_edges

[(2, 7), (1, 7)]

In [ ]:
def get_undirected_edges(x):
    result = {frozenset(i) for i in x.edges() if x.has_edge(*i[::-1])}
    result = [([*i][0], [*i][1]) for i in [*result]]
    return result

def add_directed_edge(network, edge, direction):
    """
    Add a directed edge to the network based on the specified direction.
    direction 0: Adds edge as i->j
    direction 1: Adds edge as j->i
    """
    i, j = edge
    
    if direction == 0:
        network.add_edge(i, j)
    else:
        network.add_edge(j, i)

def get_all_dags(graph):
    
    network = nx.DiGraph(graph)
    
    if nx.is_directed_acyclic_graph(network):
        dags = [network]
        
    else:
        undirected_edges = get_undirected_edges(network)
        directions = list(product([0, 1], repeat = len(undirected_edges)))
        dags = []

        for i in directions:
            
            new_network = copy.deepcopy(network)
            
            for j, k in enumerate(i):
                
                l = 
                m = 
                
                network.remove_edge(i, j)
                network.remove_edge(j, i)
                
                add_directed_edge(network, undirected_pairs[j], direction)
        
            if nx.is_directed_acyclic_graph(network):
                dags.append(network)

        # print(f"Found {len(dags)} unique DAGs within the CPDAG.")
    
    return dags

In [ ]:
dags = find_all_dags_incpdag(cpdag_graph)

In [ ]:
bic_calculator = BicScore(pd.DataFrame(data))
bics = [bic_calculator.score(i) for i in dags]

In [ ]:
print(min(bics))
print(bics.index(min(bics)))

In [ ]:
def exp_random_order(data):
    
    labels = data.columns.tolist()
    random.shuffle(labels)
    exp_data = np.array(data[labels])
    
    graph, sepetated_set = pc_algorithm.pc_get_graph(exp_data)
    cpdag = pc_algorithm.pc_apply_rules(graph, sepetated_set)
    dags = find_all_dags_incpdag(cpdag)
    # dag = find_one_dag_in_cpdag()
    
    if (len(dags)>0):
        
        bic_calculator = BicScore(pd.DataFrame(exp_data))
        bics = [bic_calculator.score(BayesianNetwork([*i.edges()])) for i in dags]
        return min(bics), labels, dags[bics.index(min(bics))]
    else:
        return None, None, None

In [ ]:
data_for_exp = utils.get_data_from_ucirepo(109)

In [ ]:
def benchmarking(data, times):
    bics = []
    orders = []
    networks = []
    
    while len(bics) <= times:
        
        bic, order, network = exp_random_order(data_for_exp)
        
        if bic is not None:
            bics.append(bic)
            orders.append(order)
            networks.append(network)

    return statistics.median(bics)

In [ ]:
benchmarking(data_for_exp, 100)

In [ ]:
def permutation_distance(seq1, seq2):
    
    index_map = {val: idx for idx, val in enumerate(seq2)}
    seq = [index_map[val] for val in seq1]

    swaps = 0
    for i in range(len(seq)):
        for j in range(len(seq) - 1):
            if seq[j] > seq[j + 1]:
                seq[j], seq[j + 1] = seq[j + 1], seq[j]
                swaps += 1

    return swaps

In [ ]:
def get_openai_embedding(x):
    
    api_key = config.get("open_ai_key_personal")
    openai_client = OpenAI(api_key=api_key)
    
    return openai_client.embeddings.create(input=x, model="text-embedding-ada-002").data

In [ ]:
texts = {
    "column_name": ["Alcohol", "Malicacid", "Ash", "Alcalinity_of_ash", "Magnesium", "Total_phenols", "Flavanoids", "Nonflavanoid_phenols", "Proanthocyanins", "Color_intensity", "Hue", "0D280_0D315_of_diluted_wines", "Proline", "class_2", "class_3"],
    "variable_name": ["Alcohol", "Malic Acid", "Ash", "Alcalinity of Ash", "Magnesium", "Total Phenols", "Flavanoids", "Nonflavanoid Phenols", "Proanthocyanins", "Color Intensity", "Hue", "OD280/OD315 of Diluted Wines", "Proline", "Class 2", "Class 3"],
    "name_description": ["Alcohol: This represents the alcohol content in the wine, usually expressed as a percentage.",
                         "Malic Acid: A naturally occurring acid in grapes and many other fruits, affecting the taste of the wine.",
                         "Ash: In winemaking, ash typically refers to the mineral residue left after the complete combustion of the wine.",
                         "Alcalinity of Ash: Measures the alkalinity (basicity) of the ash content in the wine, affecting the overall taste balance of the wine.",
                         "Magnesium: A mineral element present in wine, influencing the growth and development of the wine.",
                         "Total Phenols: Includes all types of phenolic compounds, which affect the color, taste, and aroma of the wine.",
                         "Flavanoids: A type of phenolic compound, important for the antioxidant properties and color of the wine.",
                         "Nonflavanoid Phenols: Another type of phenolic compound, also contributing to the characteristics of the wine.",
                         "Proanthocyanins: A type of flavonoid, related to the bitterness and color of the wine.",
                         "Color Intensity: Indicates the depth of color in the wine, related to the grape varieties used and the winemaking process.",
                         "Hue: Refers to the color of the wine, which can vary from pale yellow to deep red.",
                         "OD280/OD315 of Diluted Wines: A method to measure the phenolic content in wines by the absorption of light at specific wavelengths.",
                         "Proline: An amino acid important for the aroma and flavor of the wine.",
                         "Class 2: This category represents wines that are classified as 'Class 2' in the dataset. The classification is based on certain characteristics or properties of the wines, which are distinct from those in other classes. This class label helps in differentiating this group of wines from others in the dataset for analysis or predictive modeling.",
                         "Class 3: This category signifies wines that fall under the 'Class 3' classification. The attributes of wines in this class are different from those in Classes 1 and 2. This classification is crucial for distinguishing these wines for purposes such as pattern recognition, clustering, or other forms of data analysis in the context of wine characteristics."]
}

In [ ]:
def get_embeddings(text, model):
    if model == "text-embedding-ada-002":
            result = get_openai_embedding(text)
            result = [i.embedding for i in result]
    else:
        model = SentenceTransformer(model)
        result = model.encode(text).tolist()
    return result

In [ ]:
model_ids = ["all-mpnet-base-v2", "all-MiniLM-L12-v2", "all-MiniLM-L6-v2", "WhereIsAI/UAE-Large-V1", "text-embedding-ada-002"]

In [ ]:
from angle_emb import AnglE

angle = AnglE.from_pretrained('WhereIsAI/UAE-Large-V1', pooling_strategy='cls').cuda()
vec = angle.encode('hello world', to_numpy=True)
print(vec)

In [ ]:
embeddings = {}
for text_key in texts.keys():
    result = {}
    for model_id in model_ids:
        print(model_id)
        embedding_result = get_embeddings(texts[text_key], model_id)
        result[model_id] = embedding_result
    embeddings[text_key] = result

In [ ]:
def exp_model_score(data, embeddings, pdist_metric = "euclidean", times = 100):
    
    scores = {}
    
    for text_key in embeddings.keys():
        
        result = {}
        
        for model_id in embeddings[text_key].keys():
            
            print(text_key, model_id)
            
            distance_matrix = squareform(pdist(embeddings[text_key][model_id], metric = pdist_metric))
            distance_matrix[:, 0] = 0
                
            text = []
            
            while len(text) <= times:
                
                permutation = solve_tsp_local_search(distance_matrix)[0]
                order = [data.columns.tolist()[i] for i in permutation]
                array = np.array(data[order])
                
                pc = PC()
                pc.learn(array)
                graph = pc.causal_matrix
                # graph, sepetated_set = pc_algorithm.pc_get_graph(array)
                # cpdag = pc_algorithm.pc_apply_rules(graph, sepetated_set)
                dags = find_all_dags_incpdag(graph)
                
                if (len(dags)>0):
                    status = False
                    calculator = BicScore(pd.DataFrame(array))
                    bics = [calculator.score(BayesianNetwork([*i.edges()])) for i in dags]
                    text.append(min(bics))
            
            result[model_id] = statistics.median(text)
            
        scores[text_key] = result
    
    return scores

In [ ]:
pd.DataFrame(exp_model_score(data_for_exp, embeddings, "cityblock"))

In [ ]:
permutation = solve_tsp_local_search(distance_matrix)[0]
order = [data.columns.tolist()[i] for i in permutation]
array = np.array(data[order])